In [289]:
from Bio import SeqIO
import urllib.parse as up
import os
import psycopg2
import re
from Bio import Entrez
from collections import Counter
from IAP_seqs import Seq

# Lista Cenas Importantes

Analisar o que fazer com o Cds_ID

Ver o que fazer com o  ID_authors tbm n faz mt sentido

Erro mt random a povoar references

Ver parte das sequences --> Precisamos de mais espaço mas o SQL limita isso

Ligar Base de Dados

In [290]:
import mysql.connector as SQLC   #ligação Bruno

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()

Ficheiro GB 

In [291]:
id_genbank_input = input("ID NCBI")

In [292]:
handle = Entrez.efetch(db='nucleotide',retmode='txt',id=id_genbank_input, rettype = "gb") # dowload do ficheiro
record = SeqIO.read(handle, "gb")

In [293]:
# # file = "/home/sa_bruno/Documentos/GitHub/Projeto_LB/ARL15/ARL15.gb"                
# file = "Teste.gb"
# record = SeqIO.read(file, "genbank") 

# Obter todas as variáveis / Povoar Base de Dados 

# Tabela Genbank

In [294]:
# Genbank
ID = record.name
Organism = record.annotations["organism"]
Accession = record.annotations["accessions"][0]
Definition = record.description
Keywords = ""

for c in record.annotations["keywords"]:
    Keywords += c + ", "
Keywords = Keywords[0: (len(Keywords) -2)]

Sequence = str(record.seq)

In [295]:
print(ID)
print(Organism)
print(Accession)
print(Definition)
print(Keywords)
print(Sequence)

HUMPROK
Homo sapiens
L00727
Human myotonin protein kinase (Mt-PK) gene, complete cds

CCATGGCCTCTCTGCACCCCGCCTCAGGGTCAGGGTCAGGGTCATGCTGGGAGCTCCCTCTCCTAGGACCCTCCCCCCAAAAGTGGGCTCTATGGCCCTCTCCCCTGGTTTCCTGTGGCCTGGGGCAAGCCAGGAGGGCCAGCATGGGGCAGCTGCCAGGGGCGCAGCCGACAGGCAGGTGTTCGGCGCCAGCCTCTCCAGCTGCCCCAACAGGTGCCCAGGCGCTGGGAGGGCGGTGACTCACGCGGGCCCTGTGGGAGAACCAGCTTTGCAGACAGGCGCCACCAGTGCCCCCTCCTCTGCGATCCAGGAGGGACAACTTTGGGTTCTTCTGGGTGTGTCTCCTTCTTTTGTAGGTTCTGCACCCACCCCCACCCCCAGCCCCAAAGTCTCGGTTCCTATGAGCCGTGTGGGTCAGCCACCATTCCCGCCACCCCGGGTCCCTGCGTCCTTTAGTTCTCCTGGCCCAGGGCCTCCAACCTTCCAGCTGTCCCACAAAACCCCTTCTTGCAAGGGCTTTCCAGGGCCTGGGGCCAGGGCTGGAAGGAGGATGCTTCCGCTTCTGCCAGCTGCCTTGTCTGCCCACCTCCTCCCCAAGCCCAGGACTCGGGCTCACTGGTCACTGGTTTCTTTCATTCCCAGCACCCTGCTCCTCTGGCCCTCATATGTCTGGCCCTCAGTGACTGGTGTTTGGTTTTTGGCCTGTGTGTAACAAACTGTGTGTGACACTTGTTTCCTGTTTCTCCGCCTTCCCCTGCTTCCTCTTGTGTCCATCTCTTTCTGACCCAGGCCTGGTTCCTTTCCCTCCTCCTCCCATTTCACAGATGGGAAGGTGGCGGCCAAGAAGGGCCAGGCCATTCAGCCTCTGGAAAAACCTTCTCCCAACCTCCCACAGCCCCTAATGACTCTCCTGG

In [296]:
try:
        cur.execute(f"""
                 INSERT INTO Genbank (ID, Organism, Accession, Definition, Keywords)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                        (ID, Organism, Accession, Definition, Keywords))
except:
        print("Erro Povoação Genbank")

Erro Povoação Genbank


# Tabela Sequences  / Parte para Algoritmos

In [297]:
alg_seq = Seq(Sequence).count_nucleotides()

Count_A = alg_seq[0]
Count_C = alg_seq[1]
Count_G = alg_seq[2]
Count_T = alg_seq[3]

In [298]:
print(ID)
print(Sequence)
print(Count_A)
print(Count_C)
print(Count_G)
print(Count_T)

HUMPROK
CCATGGCCTCTCTGCACCCCGCCTCAGGGTCAGGGTCAGGGTCATGCTGGGAGCTCCCTCTCCTAGGACCCTCCCCCCAAAAGTGGGCTCTATGGCCCTCTCCCCTGGTTTCCTGTGGCCTGGGGCAAGCCAGGAGGGCCAGCATGGGGCAGCTGCCAGGGGCGCAGCCGACAGGCAGGTGTTCGGCGCCAGCCTCTCCAGCTGCCCCAACAGGTGCCCAGGCGCTGGGAGGGCGGTGACTCACGCGGGCCCTGTGGGAGAACCAGCTTTGCAGACAGGCGCCACCAGTGCCCCCTCCTCTGCGATCCAGGAGGGACAACTTTGGGTTCTTCTGGGTGTGTCTCCTTCTTTTGTAGGTTCTGCACCCACCCCCACCCCCAGCCCCAAAGTCTCGGTTCCTATGAGCCGTGTGGGTCAGCCACCATTCCCGCCACCCCGGGTCCCTGCGTCCTTTAGTTCTCCTGGCCCAGGGCCTCCAACCTTCCAGCTGTCCCACAAAACCCCTTCTTGCAAGGGCTTTCCAGGGCCTGGGGCCAGGGCTGGAAGGAGGATGCTTCCGCTTCTGCCAGCTGCCTTGTCTGCCCACCTCCTCCCCAAGCCCAGGACTCGGGCTCACTGGTCACTGGTTTCTTTCATTCCCAGCACCCTGCTCCTCTGGCCCTCATATGTCTGGCCCTCAGTGACTGGTGTTTGGTTTTTGGCCTGTGTGTAACAAACTGTGTGTGACACTTGTTTCCTGTTTCTCCGCCTTCCCCTGCTTCCTCTTGTGTCCATCTCTTTCTGACCCAGGCCTGGTTCCTTTCCCTCCTCCTCCCATTTCACAGATGGGAAGGTGGCGGCCAAGAAGGGCCAGGCCATTCAGCCTCTGGAAAAACCTTCTCCCAACCTCCCACAGCCCCTAATGACTCTCCTGGCCTCCCTTTAGTAGAGGATGAAGTTGGGTTGGCAGGGTAAACTGAGACCGGGTGGGGTAGGGGTCTGGCGCTCCCGGG

In [321]:
try:

        cur.execute("""
                INSERT INTO Sequences (ID, Sequence, Count_A, Count_C, Count_T, Count_G)    
                VALUES (%s, %s, %s, %s, %s,%s)
                """,
                        (ID, Sequence, Count_A, Count_C, Count_T, Count_G))   

except:
        print("Erro Povoação Sequences")

Erro Povoação Sequences


# Tabela Locus

In [300]:
# Locus 
Size = len(record.seq)
Molecular_type = record.annotations["molecule_type"] + " " + record.annotations["topology"]
Genbank_division = record.annotations["data_file_division"] 
Modification_date = record.annotations["date"]

In [301]:
print(ID)
print(Size)
print(Molecular_type)
print(Genbank_division)
print(Modification_date)

HUMPROK
11613
DNA linear
PRI
07-SEP-1995


In [302]:
try:
        cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

except:
        print("Erro Povoação Locus")

# Tabela Features

In [303]:
temp_l = []
for c in range(len(record.features)):
    temp_l.append(record.features[c].type)

y = Counter(temp_l)
Gene = y["gene"]
# Regulatory = 
#(ID, CDS, Gene, Regulatory, exons, poli_A-site, misc_feature, mRNA))      # acabar esta parte dps  easy
# so contar apartir daqui 
  # Counter({'gene': 5125, 'CDS': 4997, 'tRNA': 105, 'rRNA': 22, 'source': 1, 'tmRNA': 1})
CDS_count = y["CDS"]
Regulatory = y["regulatory"]
exons = y["exons"]
poli_A_site = y["poli_A-site"]                    #  poli_A-site
misc_feature = y["misc_feature"]
mRNA = y["mRNA"]
Location_span = str(record.features[0].location)

In [304]:
print(ID)
print(CDS_count)
print(str(CDSID) + " wwww") # ver mt bem isto dps 
print(Gene)
print(Regulatory)
print(poli_A_site)
print(misc_feature)
print(mRNA)
print(Location_span)

HUMPROK
6
0 wwww
1
0
0
3
8
[0:11613](+)


In [305]:
try:
        cur.execute("""
        INSERT INTO Features (ID, CDS_count, CDSID, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID, CDS_count, CDSID, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 
except:
        print("Erro Features")

Erro Features


# Tabela CDS

# Importante Temos de ver como fazemos com o Puto do CDS_ID

In [306]:
# usar para dps
CDS_list = []

ID_cds = []
Translacion_seq = []
Localization = []
ID_protein = []
Protein = []
for c in range(len(record.features)):
    if record.features[c].type == "CDS":
        CDS_list.append(record.features[c])

# Translacion_seq
for c in range(len(CDS_list)):
    ID_cds.append(CDS_list[c])
    Translacion_seq.append(CDS_list[c].qualifiers["translation"])
    Localization.append(CDS_list[c].location)
    ID_protein.append(CDS_list[c].qualifiers["protein_id"])
    try:
        Protein.append(CDS_list[c].qualifiers["product"])
    except:
        Protein.append("------")
    


In [307]:
for c in range(len(ID_cds)):
    print(ID_cds[c])
    print()
    print(Translacion_seq[c])
    print()
    print(Localization[c])
    print()
    print(ID_protein[c])
    print()
    print(Protein[c])
    print()
    print()


type: CDS
location: join{[1853:2135](+), [2390:2474](+), [2553:2649](+), [3271:3420](+), [3692:3786](+), [4039:4246](+), [4321:4585](+), [5900:5986](+), [8183:8295](+), [9007:9165](+), [9270:9368](+), [9539:9586](+), [9875:9965](+), [10295:10448](+)}
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['Mt-PK']
    Key: note, Value: ['Form I mRNA']
    Key: product, Value: ['myotonin-protein kinase, Form I']
    Key: protein_id, Value: ['AAA75236.1']
    Key: translation, Value: ['MGGHFWPPEPYTVFMWGSPWEADSPRVKLRGREKGRQTEGGAFPLVSSALSGDPRFFSPTTPPAEPIVVRLKEVRLQRDDFEILKVIGRGAFSEVAVVKMKQTGQVYAMKIMNKWDMLKRGEVSCFREERDVLVNGDRRWITQLHFAFQDENYLYLVMEYYVGGDLLTLLSKFGERIPAEMARFYLAEIVMAIDSVHRLGYVHRDIKPDNILLDRCGHIRLADFGSCLKLRADGTVRSLVAVGTPDYLSPEILQAVGGGPGTGSYGPECDWWALGVFAYEMFYGQTPFYADSTAETYGKIVHYKEHLSLPLVDEGVPEEARDFIQRLLCPPETRLGRGGAGDFRTHPFFFGLDWDGLRDSVPPFTPDFEGATDTCNFDLVEDGLTAMVSGGGETLSDIREGAPLGVHLPFVGYSYSCMALRDSEVPGPTPMEVEAEQLLEPHVQAPSLEPSVSPQDETAEVAVPAAVPAAEAEAEVTLRELQEALEEEVLTRQSLSR

In [308]:
num_list = 0                            # cdc ID que falta ver
for c in range(len(ID_cds)):
        cur.execute("""
                INSERT INTO CDS (CDS_ID, Translacion_seq, Localization, ID_protein, Protein)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                (num_list, str(Translacion_seq[num_list]), str(Localization[num_list]), str(ID_protein[num_list]), str(Protein[num_list]))) 
        num_list += 1

IntegrityError: 1062 (23000): Duplicate entry '0' for key 'CDS.PRIMARY'

# Tabela References

In [313]:
def get_authors(a):
    exis_5 = re.search(r"authors:\s+([\w\s,\.-]+)", a)          #dúvida (como é que o ciclo sabe se isto está vazio ou não)       
    if exis_5:
        Authors = []
        Authors_work = []
        y = re.search(r"authors:\s+([\w\s,\.-]+)", a)    # isto aqui é o remark
        Authors_work.append(y.group(1))
        # else:
        #     Consortium.append("------")
        help_authors = [] 
        Authors = str(Authors_work[0]).split("., ")
        for i, c in enumerate(Authors):
            f = c.split(" and ")
            help_authors.append([i, f])
        c = len(Authors)
        i = 0
        while c > 0:
            if len(help_authors[i][1]) > 1:
                Authors.remove(Authors[i])
                for o in help_authors[i][1]:
                    Authors.append(o)

            c -= 1
            i += 1
    else:
        Authors = []

    return Authors



In [314]:
# referencias  # faz parte do PubMed INfo
Pubmed = []
Title = []
Journal = []
Consortium = []
Remark = []
Authors = []
b = ""
pubmed_trash = record.annotations["references"]    # resolver isto 
c = len(pubmed_trash)
i = 0
while c > 0:
    a = str(pubmed_trash[i]).replace("\n", "§")   
    
    exist_3 = re.search(r"pubmed id:\s+(\d+)", a)
    if exist_3:
        m = re.search(r"pubmed id:\s+(\d+)", a)                 # COnfirmar PubMEd haver sempre/nao
        Pubmed.append(m.group(1))
    else:
        Pubmed.append("------")

    
    n = re.search(r"title:\s+([\w\s]+)", a)
    Title.append(n.group(1))

    exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)
    if exist_2:
        j = re.search(r"journal:\s+([\w\s.()--,]+)", a)
        Journal.append(j.group(1))
    else:
        Journal.append("------")

    exis = re.search(r"consrtm:\s+([\w\s]+)", a)
    if exis:
        k = re.search(r"consrtm:\s+([\w\s]+)", a)    # isto aqui é o remark
        Consortium.append(k.group(1))
    else:
        Consortium.append("------")         # ter cuidado com isto dps
    

    exis_1 = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\s\d])", a)
    if exis_1:
        l = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\d\s:\[;\(\)\.,!?<>{}\"]+)", a)
        Remark.append(l.group(1))
    else:
        Remark.append("------") 

    Authors.append(get_authors(a))


    
    c -= 1
    i += 1

In [315]:
print(ID)
print(Pubmed)
print(Authors)
print(Title)
print(Journal)
print(Consortium)
print(Remark)


HUMPROK
['8469976', '1546326']
[['Fu,Y.-H', 'Friedman,D.L', 'Richards,S', 'Pearlman,J.A', 'Gibbs,R.A', 'Pizzuti,A', 'Ashizawa,T', 'Perryman,M.B', 'Scarlato,G', 'Fenwick,R.G. Jr.', 'Caskey,C.T.'], ['Fu,Y.-H', 'Pizzuti,A', 'Fenwick,R.G. Jr', 'King,J', 'Rajnarayan,S', 'Dunne,P.W', 'Dubel,J', 'Nasser,G.A', 'Ashizawa,T', 'de Jong,P', 'Wieringa,B', 'Korneluk,R', 'Perryman,M.B', 'Epstein,H.F.', 'Caskey,C.T.']]
['Decreased expression of myotonin', 'An unstable triplet repeat in a gene related to myotonic muscular dystrophy']
['Science 260 (5105), 235-238 (1993)', 'Science 255 (5049), 1256-1258 (1992)']
['------', '------']
['------', '------']


In [316]:
reference_count = 0
for c in Pubmed:    
        cur.execute("""
                INSERT INTO References(ID, Pubmed, Authors, Title, Journal, Consortium, Remark)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """,
                (ID, str(Pubmed[reference_count]), str(Authors[reference_count]), str(Title[reference_count]), str(Journal[reference_count]), str(Consortium[reference_count]), str(Remark[reference_count])))
        reference_count += 1

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'References(ID, Pubmed, Authors, Title, Journal, Consortium, Remark)
            ' at line 1

# Tabela Pubmed_Info

In [317]:
Abstract = []                        # mudar para cada referenias 
DOI = []
Affiliation = []
for c in Pubmed:
    if c != "------":
        Entrez.email = 'sa.bruno.2001@gmail.com'
        # handle = Entrez.efetch(db='pubmed',retmode='xml',id="15496913")
        handle = Entrez.efetch(db='pubmed',retmode='xml',id=c)
        resultado = Entrez.read(handle)
        q = str(resultado["PubmedArticle"][0])

        o = re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q)
        Abstract.append(o.group(1))

        p = re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q)
        DOI.append(p.group(1))

        if re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q):
            t = re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q)
            Affiliation.append(t.group(1))
        else:
            Affiliation.append("None")


In [318]:
print(Title)
print(Abstract)
print(DOI)

['Decreased expression of myotonin', 'An unstable triplet repeat in a gene related to myotonic muscular dystrophy']
['The myotonic dystrophy mutation has recently been identified', "Synthetic oligonucleotides containing GC-rich triplet sequences were used in a scanning strategy to identify unstable genetic sequences at the myotonic dystrophy (DM) locus. A highly polymorphic GCT repeat was identified and found to be unstable, with an increased number of repeats occurring in DM patients. In the case of severe congenital DM, the paternal triplet allele was inherited unaltered while the maternal, DM-associated allele was unstable. These studies suggest that the mutational mechanism leading to DM is triplet amplification, similar to that occurring in the fragile X syndrome. The triplet repeat sequence is within a gene (to be referred to as myotonin-protein kinase), which has a sequence similar to protein kinases.'"]
['10.1126/science.8469976', '10.1126/science.1546326']


In [319]:
pubmed_count= 0
for c in Pubmed:
    cur.execute("""
            INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI)   
            VALUES (%s, %s, %s, %s)
            """,
            (Pubmed[pubmed_count], Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count]))   
if c != "------":
        pubmed_count += 1

IntegrityError: 1062 (23000): Duplicate entry '8469976' for key 'Pubmed_Info.PRIMARY'

# Lixo para baixo

In [ ]:

    
#     cur.execute("""
#         INSERT INTO References (ID, Pubmed, Title, Journal, Consortium, Remark)     # saltamos
#         VALUES (%s, %s, %s, %s, %s, %s))
#         """,
#         (ID, Pubmed, Journal, Consortium, Remark))   
    
#     cur.execute("""
#         INSERT INTO Authors (ID_authors, Affiliation)    
#         VALUES (%s, %s, %s))
#         """,
#         (Authors, Affiliation))    